# Install dependencies

In [ ]:
# !pip uninstall terraform -y
# !pip uninstall terraform-gpu -y
!pip install tensorflow==2.3.0
!pip install tensorflow-gpu==2.3.0

# Imports

In [ ]:
import os, gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from shutil import copyfile

# Set up constants
Set a batch size of 32

Set the image height and width (constant for KDEF dataset)

In [ ]:
batchSize = 1
imgHeight = 762
imgWidth = 562

# Fetch Dataset
* Mount drive


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


* Copy data from drive

In [ ]:
# Create dataset directory if it does not exist
dataDir = "/content/dataset"
if not (os.path.exists(dataDir)):
  os.makedirs(dataDir)

# Create emotion directories (if they don't exist)
emotionCodes = ["AF", "AN", "DI", "HA", "NE", "SA", "SU"]
for emotion in emotionCodes:
  if not (os.path.exists(os.path.join(dataDir, emotion))):
    os.makedirs(os.path.join(dataDir, emotion))

# Count files (so we can guess how long it will take to copy over)
fileCount = 0
for root, subDirs, files in os.walk("/content/gdrive/MyDrive/KDEF"):
  for file in files:
    fileCount +=1
print(f"Copying {fileCount} files...")
fileCount = 0 # Reusing filecount variable

# Copy files across from Google drive
for root, subDirs, files in os.walk("/content/gdrive/MyDrive/KDEF"):
  for file in files:
    fileCount += 1
    if len(file) == 11: # Straight ahead image
      imageDir = "S"
      emotionCode = file[4:-5]
    else:
      imageDir = file[6:-4]
      emotionCode = file[4:-6]
    copyfile(os.path.join(root, file), os.path.join(dataDir, emotionCode, file))
    print(f"\rCopied {fileCount} images so far", end="")
print("\nSuccessfully copied across all images")

Copying 2938 files...
Copied 2938 images so far
Successfully copied across all images


# Create Dataset

In [ ]:
dataSetDir = "/content/dataset"
print(f"Creating dataset from {dataSetDir} directory...")
trainingDataset = tf.keras.preprocessing.image_dataset_from_directory(
  dataSetDir,
  validation_split=0.2, # 20% of images will be used for validation
  subset="training",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize
)
validationDataset = tf.keras.preprocessing.image_dataset_from_directory(
  dataSetDir,
  validation_split=0.2, # 20% of images will be used for validation
  subset="validation",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize
)
print(f"Found class names {trainingDataset.class_names}")

# Configure dataset for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
trainingDataset = trainingDataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validationDataset = validationDataset.cache().prefetch(buffer_size=AUTOTUNE)

# Standardize data (rescale rgb values from 0-255 to 0-1)
normalizationLayer = layers.experimental.preprocessing.Rescaling(1./255)

# Create model (model shape and size to be investigated, maybe improved)

classNo = 7
model = keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(imgHeight, imgWidth, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(classNo)    
])

# Compile model
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
)
print("Model summary:")
print(model.summary())

Creating dataset from /content/dataset directory...
Found 2938 files belonging to 7 classes.
Using 2351 files for training.
Found 2938 files belonging to 7 classes.
Using 587 files for validation.
Found class names ['AF', 'AN', 'DI', 'HA', 'NE', 'SA', 'SU']
Model summary:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 762, 562, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 762, 562, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 381, 281, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 381, 281, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 190, 140, 3

# Train model

In [ ]:
# Manually run garbage collection to clear RAM
gc.collect()

# Train model
epochs=5
history = model.fit(
  trainingDataset,
  validation_data=validationDataset,
  epochs=epochs
)

Epoch 1/5
1155/2351 [=============>................] - ETA: 13:32 - loss: 2.2389 - accuracy: 0.1229